# **Práctica #1: Modelo para predicción de lluvia**

*Centro Universitario de Ciencias Exactas e Ingenierías*

*División de Tecnologías para la Integración Ciber-Humana*

*Ingeniería Biomédica*

<br>

*Mtra. Sofía Alejandra Aguilar Valdez*

2 de septiembre de 2022

## **Información del equipo**

```NOMBRES:``` \\
Victor Alvarado Aparicio \\
Dayana Analy Pacheco Bañuelos \\
Leonardo Juàrez Zucco

```CÓDIGOS:``` \\
215767685 \\
######### \\
220285869

```LINK REPOSITORIO:``` https://github.com/L-Rittmeister/U1_Team.git



## **Contenido**



1.   Resumen
2.   Marco teórico
3.   Objetivos
4.   Materiales y métodos
5.   Resultados
6.   Discusión
7.   Conclusiones
8.   Referencias



# **1. Resumen**
(100-300 palabras)

# **2. Marco teórico**
(300-800 palabras)

# **3. Objetivos**
Objetivo general y objetivos específicos

# **4. Materiales y métodos**

## *Materiales*

Describir este conjunto de datos [[1]](https://www.kaggle.com/code/fatmakursun/rain-forecasting-with-artificial-neural-network/data).

## *Métodos*

1. Esquema de metodología
2. Descripción de los métodos y su implementación en código en forma de narrativa.

In [2]:
#Mounting drive storage to access data (if needed/running on collab then uncomment)
# Mount Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

In [3]:
# Import Libraries
import pandas as pd
import numpy as np
from numpy import sqrt
from numpy import vstack
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, confusion_matrix, classification_report
from sklearn.metrics import roc_curve
from tqdm.notebook import trange, tqdm
import time
import torch
from torch import nn
from torch.nn.functional import normalize
from torchvision import datasets, transforms
import torch.utils.data as data
from torch.utils.data import Dataset
import torch.nn.functional as F
from torch import optim
from torch.optim import SGD
from torchvision import datasets, transforms
from torch.utils.data import random_split, DataLoader
from torch.nn.modules.activation import ReLU
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_

In [4]:
#Pytorch and CUDA installation test
torch.randn(5).cuda()

tensor([-0.1085,  0.2888,  1.9000, -1.8322,  0.1047], device='cuda:0')

In [5]:
#Load the dataset on a pandas dataframe
Data = pd.read_csv("seattleWeather_1948-2017.csv")
Data

,DATE,PRCP,TMAX,TMIN,RAIN
0,1948-01-01,0.47,51,42,True
1,1948-01-02,0.59,45,36,True
2,1948-01-03,0.42,45,35,True
3,1948-01-04,0.31,45,34,True
4,1948-01-05,0.17,45,32,True
...,...,...,...,...,...
25546,2017-12-10,0.00,49,34,False
25547,2017-12-11,0.00,49,29,False
25548,2017-12-12,0.00,46,32,False
25549,2017-12-13,0.00,48,34,False


In [7]:
#Drop NaN value
Data.dropna(inplace=True)
Data.reset_index(inplace=True, drop=True)

C:\Users\Lordleojz\AppData\Local\Temp\ipykernel_26248\4122486251.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data.dropna(inplace=True)


In [8]:
#Create a n array with the values of PRCP, TMAX, TMIN, and RAIN
Data = Data[['PRCP', 'TMAX', 'TMIN', 'RAIN']]
Data

,PRCP,TMAX,TMIN,RAIN
0,0.47,51,42,True
1,0.59,45,36,True
2,0.42,45,35,True
3,0.31,45,34,True
4,0.17,45,32,True
...,...,...,...,...
25543,0.00,49,34,False
25544,0.00,49,29,False
25545,0.00,46,32,False
25546,0.00,48,34,False


In [9]:
#Eliminate possible void values
Data = Data[Data['PRCP'] != 'T']
Data = Data[Data['TMAX'] != 'T']
Data = Data[Data['TMIN'] != 'T']
Data = Data[Data['RAIN'] != 'T']
Data

,PRCP,TMAX,TMIN,RAIN
0,0.47,51,42,True
1,0.59,45,36,True
2,0.42,45,35,True
3,0.31,45,34,True
4,0.17,45,32,True
...,...,...,...,...
25543,0.00,49,34,False
25544,0.00,49,29,False
25545,0.00,46,32,False
25546,0.00,48,34,False


In [10]:
#Convert the values of PRCP, TMAX, TMIN, and RAIN to float
Data['PRCP'] = Data['PRCP'].astype(float)
Data['TMAX'] = Data['TMAX'].astype(float)
Data['TMIN'] = Data['TMIN'].astype(float)
Data['RAIN'] = Data['RAIN'].astype(float)
Data

,PRCP,TMAX,TMIN,RAIN
0,0.47,51.0,42.0,1.0
1,0.59,45.0,36.0,1.0
2,0.42,45.0,35.0,1.0
3,0.31,45.0,34.0,1.0
4,0.17,45.0,32.0,1.0
...,...,...,...,...
25543,0.00,49.0,34.0,0.0
25544,0.00,49.0,29.0,0.0
25545,0.00,46.0,32.0,0.0
25546,0.00,48.0,34.0,0.0


In [11]:
#Create the tensor
Data = torch.tensor(Data.values)

In [12]:
#Split the tensor into training and testing
Train = Data[:int(0.8*len(Data))]
Test = Data[int(0.8*len(Data)):]

In [13]:
#Create the training and testing dataloaders
Train_loader = DataLoader(Train, batch_size=32, shuffle=True)
Test_loader = DataLoader(Test, batch_size=32, shuffle=True)

In [14]:
#Create the model
class Predictor(nn.Module):
    def __init__(self):
        super(Predictor, self).__init__()
        self.l1=nn.Linear(4, 4)
        self.l2=nn.Linear(4, 4)
        self.l3=nn.Linear(4, 1)
        self.do=nn.Dropout(0.1)
    def forward(self, x):
        h1 = nn.functional.relu(self.l1(x))
        h2 = nn.functional.relu(self.l2(h1))
        do = self.do(h2+h1)
        logits = self.l3(do)
        return logits

In [15]:
Model = Predictor().cuda()

In [16]:
#Define cost and optimizer
Parameters = Model.parameters()
Optimizer = torch.optim.SGD(Parameters, lr=0.001)
Loss = nn.CrossEntropyLoss()

# **5. Resultados**

# **6. Discusión**
Underfitting, overfitting, o no y el porqué
Comparación de su metodología con literatura asociada

# **7. Conclusiones**
(100-300 palabras).

# **8. Referencias**

[1] Pb, V. (2020, February 18). Perceptron. Kaggle. Retrieved June 1, 2022, from https://www.kaggle.com/code/prashfio/perceptron/notebook